In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=False)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)


net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01)


num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 200 == 199:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')


correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network: %d %%' % (100 * correct / total))


Files already downloaded and verified
Files already downloaded and verified


C:\Users\vsonu\anaconda3\Lib\site-packages\torchvision\transforms\functional.py:79: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  img = torch.ByteTensor(torch.ByteStorage.from_buffer(pic.tobytes()))


[1,   200] loss: 2.298
[1,   400] loss: 2.282
[1,   600] loss: 2.210
[2,   200] loss: 1.973
[2,   400] loss: 1.887
[2,   600] loss: 1.834
[3,   200] loss: 1.707
[3,   400] loss: 1.663
[3,   600] loss: 1.614
[4,   200] loss: 1.540
[4,   400] loss: 1.513
[4,   600] loss: 1.502
[5,   200] loss: 1.446
[5,   400] loss: 1.437
[5,   600] loss: 1.407
[6,   200] loss: 1.369
[6,   400] loss: 1.330
[6,   600] loss: 1.347
[7,   200] loss: 1.290
[7,   400] loss: 1.294
[7,   600] loss: 1.263
[8,   200] loss: 1.217
[8,   400] loss: 1.214
[8,   600] loss: 1.207
[9,   200] loss: 1.159
[9,   400] loss: 1.153
[9,   600] loss: 1.139
[10,   200] loss: 1.099
[10,   400] loss: 1.106
[10,   600] loss: 1.075
Finished Training
Accuracy of the network on the 10000 test images: 56 %


In [8]:
#Incorporating Batch Normalization and regularization loss to the model to improve its performance


class CNN_batchnorm(nn.Module):
    def __init__(self):
        super(CNN_batchnorm, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool(torch.relu(self.bn2(self.conv2(x))))
        x = self.pool(torch.relu(self.bn3(self.conv3(x))))
        x = x.view(-1, 128 * 4 * 4)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
def reg(model, reg_strength):
    reg_loss = 0
    for param in model.parameters():
        reg_loss += torch.norm(param, p=2)  
    return reg_strength * reg_loss

CNN_batch = CNN_batchnorm()

optimizer_batch = optim.SGD(CNN_batch.parameters(), lr=0.01)


reg_strength = 0.001
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer_batch.zero_grad()
        outputs = CNN_batch(inputs)
        reg_loss = reg(CNN_batch,reg_strength)
        loss = criterion(outputs, labels) + reg_loss
        loss.backward()
        optimizer_batch.step()
        running_loss += loss.item()
        if i % 200 == 199:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')


correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        outputs = CNN_batch(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network with batch normalization and regulazization: %d %%' % (100 * correct / total))

[1,   200] loss: 1.826
[1,   400] loss: 1.478
[1,   600] loss: 1.339
[2,   200] loss: 1.145
[2,   400] loss: 1.104
[2,   600] loss: 1.062
[3,   200] loss: 0.969
[3,   400] loss: 0.967
[3,   600] loss: 0.941
[4,   200] loss: 0.856
[4,   400] loss: 0.848
[4,   600] loss: 0.863
[5,   200] loss: 0.790
[5,   400] loss: 0.782
[5,   600] loss: 0.789
[6,   200] loss: 0.720
[6,   400] loss: 0.732
[6,   600] loss: 0.716
[7,   200] loss: 0.660
[7,   400] loss: 0.665
[7,   600] loss: 0.672
[8,   200] loss: 0.599
[8,   400] loss: 0.617
[8,   600] loss: 0.626
[9,   200] loss: 0.571
[9,   400] loss: 0.566
[9,   600] loss: 0.570
[10,   200] loss: 0.523
[10,   400] loss: 0.523
[10,   600] loss: 0.539
Finished Training
Accuracy of the network with batch normalization and regulazization: 69 %
